In [1]:
import os
import tweepy as tw #be sure to "pip install tweepy"
import numpy as np
import pandas as pd
import time
from IPython import display

In [2]:
#import necessary keys from Twitter Developer API
CONSUMER_KEY = 'uUxsn1nVeotKTGiMXFe9ZV058'
CONSUMER_SECRET = 'EsKP4gYXWEeLjKc4CUBw5gwkzJnXut0kWeqM6RS0GqQ23uQzDX'
ACCESS_TOKEN = '1222371563594506240-usNlkp4rMMjc6E5r2Rab0A6eovPkPi'
ACCESS_TOKEN_SECRET = 'N6xLlu5KDcGMhNsJxYw47JRNAA7Q7B6XcYjyNTrRnaAka'

In [3]:
auth = tw.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tw.API(auth, wait_on_rate_limit=True)#, wait_on_rate_limit_notify=True)

In [4]:
#DOUBLE CHECK TO MAKE SURE YOU'RE NOT HITTING CURSOR RATE LIMIT (see screenshot from tweepy docs)
#multithreading?

In [11]:
def metadata_df(tweets_cursor, lat_long_only=True):
    # create empty list
    tweet_list =[]
    total_tweets = 0
    latlong_tweets = 0
    
    # loop through tweet objects
    for tweet in tweets_cursor:
        #this just prints the tweets so we know the progress of pulling
        total_tweets += 1
        print("Total tweets pulled: ",total_tweets)
        print("Tweets with valid latitude/longitude: ",latlong_tweets)
        display.clear_output(wait=True)
        
        #NOTE: this line ensures that we only return tweets with valid locations/latitudes + longitudes
        if lat_long_only:
            if not tweet.place: continue
            
        #first, get location (city) and country depending on what info is available
        if tweet.place != None:
            location = tweet.place.full_name
            country = tweet.place.country_code
            
        else:
            location = tweet.user.location #if actual city doesn't exist, get location provided by the user
            country = np.nan
            
        #get latitude and longitude depending on whether an exact location or bounding box is provided
        if tweet.coordinates != None: #exact location
            longitude, latitude = tweet.coordinates['coordinates']
            
        elif tweet.place != None: #bounding box --> take center point (average) of box
            bounding_box = np.array(tweet.place.bounding_box.coordinates[0])
            longitude, latitude = np.mean(bounding_box, axis=0)
            
        else: #no location provided --> make nan
            longitude = np.nan
            latitude = np.nan
        
        # append attributes to list
        tweet_list.append({'tweet_id':tweet.id,
                           'created_at':tweet.created_at,
                           'user_id':tweet.user.id,
                           'user_name':tweet.user.screen_name,
                           'location':location,
                           'country':country,
                           'latitude':latitude,
                           'longitude':longitude,
                           'text':tweet.full_text,
                          })
        latlong_tweets += 1
        print("Total tweets pulled: ",total_tweets)
        print("Tweets with valid latitude/longitude: ",latlong_tweets)
        display.clear_output(wait=True)

    # create dataframe   
    df = pd.DataFrame(tweet_list, columns=['tweet_id',
                                           'created_at',
                                           'user_id',
                                           'user_name',
                                           'location',
                                           'country',
                                           'latitude',
                                           'longitude',
                                           'text'])
    print("Total tweets pulled: ",total_tweets)
    print("Tweets with valid latitude/longitude: ",latlong_tweets)
    return df

In [10]:
#collect tweets
tweets = tw.Cursor(api.search,
              q='#coronavirus OR #covid19 -filter:retweets',
              #lang="en",
              since='2020-03-01',
              tweet_mode='extended').items(500) #returns Cursor() object, like an iterator
df = metadata_df(tweets,lat_long_only=True)
df

Total tweets pulled:  500
Tweets with valid latitude/longitude pulled:  22


,tweet_id,created_at,user_id,user_name,location,country,latitude,longitude,text
0,1239273557655461888,2020-03-15 19:33:35,865998476248829952,LabPlazaEgana,"Santiago, Chile",CL,-33.452161,-70.659615,Prevengamos todos el #coronavirus #COVIDー19 \n...
1,1239273537980059649,2020-03-15 19:33:30,531460977,Quique_Blanes,"Alcoy, Spain",ES,38.681580,-0.531210,Esto no va conmigo. No tengo factores de riesg...
2,1239273526898589698,2020-03-15 19:33:28,386442199,HibbsLupusTrust,"Cannock, England",GB,52.699658,-2.008080,When you have a health condition like #lupus y...
3,1239273522922389505,2020-03-15 19:33:27,14295985,mdemeulder,"Antwerp, Belgium",BE,51.260556,4.358190,No interpreters yet for #coronavirus press con...
4,1239273500591960071,2020-03-15 19:33:21,477257898,scwolf5,"Aguascalientes, México",MX,21.849459,-102.327207,Primer caso confirmado del #coronavirus en #Ag...
5,1239273497488060416,2020-03-15 19:33:21,1485192834,RTVECantabria,"Cantabria, España",ES,43.136235,-4.001453,🔴| ÚLTIMA HORA @saludcantabria confirma 1 nuev...
6,1239273492425703425,2020-03-15 19:33:19,184001213,sfabrmoy,"Alicante, Spain",ES,38.348905,-0.515521,Pues nada...si seguimos en casa y siguen entra...
7,1239273491221712898,2020-03-15 19:33:19,39426554,CorkRebel69,Cork,IE,51.903863,-9.033307,"Just to show how seriously we, as a nation, ar..."
8,1239273487644200960,2020-03-15 19:33:18,256922434,jarm_cartagena,"Cartagena, Spain",ES,37.632516,-0.956809,Gracias a los que luchan contra el #coronaviru...
9,1239273485962117122,2020-03-15 19:33:18,1290098222,Mr_Matt_Beltran,"Visalia, CA",US,36.319486,-119.319526,Can we all just go back to fighting over @Pope...
